In [1]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

INPUT_DATA='PATH'
OUTPUT_DATA='PATH.npy'

#测试数据与验证数据比例
VALIDATION_PERCENTAGE=10
TEST_PERCENTAGE=10

#数据处理，读取数据并将数据分割成训练数据，验证数据和测试数据
def create_image_lists(sess,testing_percentage,validation_percentage):
    sub_dirs=[x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir=True
    
    #初始化
    training_images=[]
    training_labels=[]
    testing_images=[]
    testing_labels=[]
    validation_images=[]
    validation_labels=[]
    current_label=0
    
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir=False
            continue
            
        extensions=['jpg','jpeg','JPG','JPEG']
        file_list=[]
        dir_neme=os.path.join(INPUT_DATA,dir_name,'*.'+extensions)
        file_list.extend(glob.glob(file_glob))
        if not file_list: continue
            
        for file_name in file_list:
            image_raw_data=gfile.FastGFile(file_name,'rb').read()
            image=tf.image.decode_jpeg(image_raw_data)
            if image.dtype!=tf.float32:
                image=tf.image.convert_image_dtype(image,dtype=tf.float32)
                image=tf.image.resize_images(image,[299,299])
                image_value=sess.run(image)
                
                chance=np.random.randint(100)
                if chance<validation_percentage:
                    validation_images.append(image_value)
                    validation_labels.append(current_label)
                    
                elif chance<(testing_percentage+validation_percentage):
                    testing_images.append(image_value)
                    testing_labels.append(current_label)
                    
                else:
                    training_images.append(image_value)
                    training_labels.append(current_label)
                    
                current_label+=1
                
        state=np.random.get_state()
        np.random.shuffle(training_images)
        np.random.set_state(state)
        np.random.shuffle(training_labels)
        
        return np,asarray([training_images,training_labels,validation_images,validation_labels,testing_images,testing_labels])

def main():
    with tf.Session() as sess:
        processed_data=create_image_lists(sess,TEST_PERCENTAGE<VALIDATION_PERCENTAGE)
        np.save(OUTPUT_FILE,processed_data)
        
if __name__=='__main__':
    main()
    
    
    
    

In [ ]:
#数据预处理好了之后，下载预训练好的inceptionv3模型
#当模型和新数据集准备好了以后，开始迁移学习

import glob
import os.path
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.platform import gfile

import tensorflow.contrib.slim.python.slim.nets.inception_v3 as inception_v3

INPUT_DATA='PATH.npy'
TRAIN_FILE='保存训练好的模型路径'
CKPT_FILE='下载后的模型的路径'

LEARNING_RATE=0.0001
STEPS=300
BATCH=32
N_CLASSES=5

